# Import Packages

In [ ]:
import rasterio
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

# Downscale

In [ ]:
# Register GDAL format drivers and configuration options with a context manager.
def downscale_geo_tiff_by_factor(input_img_path,output_img_path,scale_factor=3):
    with rasterio.Env():
        with rasterio.open(input_img_path) as input_dataset:
            # scale image transform
            transform = input_dataset.transform * input_dataset.transform.scale(
                scale_factor,  # rows
                scale_factor  # cols
            )
            out_shape=(int(input_dataset.height/scale_factor), int(input_dataset.width/scale_factor))
            profile = input_dataset.profile
            profile.update(transform=transform, width=out_shape[1], height=out_shape[0])
            band_number = input_dataset.read().shape[0]
            with rasterio.open(output_img_path, 'w', **profile) as output_dataset:
                for i in range(band_number):
                    data = input_dataset.read(i+1, out_shape=out_shape, resampling=Resampling.bilinear)
                    output_dataset.write(data, i+1)

In [ ]:
# Input file name structure
data_path = "../raw_data/wc2021-2100/"
feature_names = ["tmin","tmax","prec","bioc"]
gcm_model_name = "ACCESS-CM2"
ssp_model_names = ["ssp126","ssp245","ssp370","ssp585"] # ["ssp126","ssp245","ssp370","ssp585"]
year_ranges = ["2061-2080","2081-2100"] # ["2021-2040","2041-2060","2061-2080","2081-2100"]

In [ ]:
%%time
# Downscale multi-band geotifs
for feature_name in feature_names:
    for ssp_model_name in ssp_model_names:
        for year_range in year_ranges:
            file_name = 'wc2.1_10m_'+feature_name+'_'+gcm_model_name+'_'+ssp_model_name+'_'+year_range+'.tif'
            input_path = data_path+file_name
            output_path = input_path[:-4]+"_downscaled"+input_path[-4:]
            downscale_geo_tiff_by_factor(input_path,output_path)

# Opening, Comparing and Testing

In [ ]:
# Downscale test on 'wc2.1_10m_bioc_ACCESS-CM2_ssp126_2021-2040.tif'
input_img_path = '../raw_data/wc2021-2100/wc2.1_10m_bioc_ACCESS-CM2_ssp245_2081-2100.tif'
output_img_path = input_img_path[:-4]+"_downscaled"+input_img_path[-4:]
# downscale_geo_tiff_by_factor(input_img_path,output_img_path)

# Check shapes
with rasterio.open(input_img_path) as input_img_dataset:
        input_img_read = input_img_dataset.read()

with rasterio.open(output_img_path) as output_img_dataset:
        output_img_read = output_img_dataset.read()

print(input_img_read.shape,"\n",output_img_read.shape)

In [ ]:
# Plot input / output
fig, axs = plt.subplots(3,2, facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
fig.set_figheight(8)
fig.set_figwidth(10)
axs = axs.ravel()

axs[0].imshow(input_img_read[0,:,:])
axs[0].set_title("Input dataset, Band 1")
axs[1].imshow(output_img_read[0,:,:])
axs[1].set_title("Output dataset, Band 1")

axs[2].imshow(input_img_read[1,:,:])
axs[2].set_title("Input dataset, Band 2")
axs[3].imshow(output_img_read[1,:,:])
axs[3].set_title("Output dataset, Band 2")

axs[4].imshow(input_img_read[2,:,:])
axs[4].set_title("Input dataset, Band 3")
axs[5].imshow(output_img_read[2,:,:])
axs[5].set_title("Output dataset, Band 3");

In [ ]:
# Notebook file size
%ls -alth 2_future_climate_data_downscale.ipynb

# Comparing feature values of two datasets

In [ ]:
# Getting feature values from spatial data sets
# The following function retrieves the values for given lat/lon
def getvalue(data, lat, lon, band=0):
    data_array = data.read()[band]
    # since the raster is in regular lon/lat grid, we can use 
    # `data.index()` to identify the index of a given lon/lat pair
    idx = data.index(lon, lat, precision=1E-6)    
    return data_array[idx]

In [ ]:
plant_data_inkl_bioclim = pd.read_csv("../raw_data/plant_data_inkl_bioclim/plant_data_inkl_bioclim_101.csv")
test_coordinates = plant_data_inkl_bioclim.sample(n=10)[["decimalLatitude","decimalLongitude"]]

In [ ]:
with  rasterio.open(input_img_path) as input_img:
    with rasterio.open(output_img_path) as output_img:
        test_coordinates["bioc01_original"] = getvalue(
            input_img,test_coordinates["decimalLatitude"],test_coordinates["decimalLongitude"])
        test_coordinates["bioc01_downsampled"] = getvalue(
            output_img,test_coordinates["decimalLatitude"],test_coordinates["decimalLongitude"])
        
        test_coordinates["bioc07_original"] = getvalue(
            input_img,test_coordinates["decimalLatitude"],test_coordinates["decimalLongitude"],band=6)
        test_coordinates["bioc07_downsampled"] = getvalue(
            output_img,test_coordinates["decimalLatitude"],test_coordinates["decimalLongitude"],band=6)
        
test_coordinates

# Other downscaling methods

In [ ]:
# !pip install tifffile
# !pip install scikit-image
# !pip install imagecodecs
# from tifffile import imread, imwrite
# from skimage.transform import resize
# import imagecodecs

# data = imread(data_path+'wc2.1_10m_bioc_ACCESS-CM2_ssp126_2021-2040.tif')

# resized_data = resize(data, (375, 500, 21))
# imwrite('multi-channel_resized.tif', resized_data, planarconfig='CONTIG')